## Importações e Configuração Inicial
 - AJUSTAR PARA BUSCAR INFORMAÇÕES DE PONTUAÇÕES MENSAIS

In [9]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
from pathlib import Path
from bs4 import BeautifulSoup
import json, re
import unicodedata
import time
import requests

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [10]:
from bs4 import BeautifulSoup
from pathlib import Path
import json, re
import unicodedata

# ajuste se precisar
HTML_PATH = Path("pagina_liga_classica.html")

def clean_name(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00a0", " ")
    s = re.sub(r"[ \t\r\n\f\v]+", " ", s).strip()
    return s

# carrega html
html = HTML_PATH.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "html.parser")

# 1) principais seletores desta vers?o da p?gina
seletores = [
    "div.info-time h3",              # novo layout (print do DevTools)
    # "a.button-container-card h3",    # h3 dentro do link do time
    # "div.card-row-info h3",          # varia?es
    # "section.container-lista-times h3",
    # "ranking-de-liga h3",
]

nomes = []
for sel in seletores:
    for el in soup.select(sel):
        t = clean_name(el.get_text(" ", strip=True))
        if t:
            nomes.append(t)

# 2) fallback: pegar o texto do link que aponta para #!/time/<id>
if not nomes:
    for a in soup.select("a[href*='#!/time/']"):
        t = clean_name(a.get_text(" ", strip=True))
        if t:
            nomes.append(t)

# 3) fallback final: regex focada no bloco info-time -> <h3>...</h3>
if not nomes:
    for m in re.findall(r'class="info-time"[^>]*>.*?<h3[^>]*>(.*?)</h3>', html, flags=re.I|re.S):
        t = BeautifulSoup(m, "html.parser").get_text(" ", strip=True)
        t = clean_name(t)
        if t:
            nomes.append(t)

# dedup (ordem de apari?o) + ordena?o alfab?tica
visto, times = set(), []
for n in nomes:
    if n and n not in visto:
        visto.add(n)
        times.append(n)

times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times[:10])  # amostra

# salva snippet JS ao lado do HTML
OUT_JS = HTML_PATH.with_name("participantesLiga_Classica.js")
OUT_JS.write_text(
    "window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n",
    encoding="utf-8"
)
print(f"Snippet salvo em: {OUT_JS}")


53 times encontrados
['Acessar o perfil do time A Lenda Super Vasco F.c', 'Acessar o perfil do time A Lenda Super Vascão f.c', 'Acessar o perfil do time AZURRA82', 'Acessar o perfil do time Bandoleros FCS', 'Acessar o perfil do time BordonFC04', 'Acessar o perfil do time C.A. Charru@', 'Acessar o perfil do time cartola scheuer17', 'Acessar o perfil do time CARTOLEIRO DO VALLE PRO26.5', 'Acessar o perfil do time dasdoresfc', 'Acessar o perfil do time DM Studio']
Snippet salvo em: participantesLiga_Classica.js


In [11]:
# ? ajuste aqui se precisar
HTML_PATH = Path("pagina_liga_classica.html")  

def clean_name(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00a0", " ")
    s = re.sub(r"[ \t\r\n\f\v]+", " ", s).strip()
    return s

# carrega html
soup = BeautifulSoup(HTML_PATH.read_text(encoding="utf-8", errors="ignore"), "html.parser")

# coleta nomes (principal + fallback)
nomes = [clean_name(el.get_text(" ", strip=True)) for el in soup.select("p.nome-time") if el]
if not nomes:
    nomes = [
        clean_name((a.get("title") or a.get_text(" ", strip=True)))
        for a in soup.select("a.link-perfil-time")
        if (a.get("title") or a.get_text(strip=True))
    ]

# dedup + ordena alfab?tico
visto, times = set(), []
for n in nomes:
    if n and n not in visto:
        visto.add(n); times.append(n)
times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times)

# snippet JS ao lado do HTML
OUT_JS = HTML_PATH.with_name("participantesLiga_Classica.js")
OUT_JS.write_text("window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n", encoding="utf-8")
print(f"Snippet salvo em: {OUT_JS}")


52 times encontrados
['A Lenda Super Vasco F.c', 'A Lenda Super Vascão f.c', 'AZURRA82', 'Bandoleros FCS', 'BordonFC04', 'C.A. Charru@', 'cartola scheuer17', 'CARTOLEIRO DO VALLE PRO26.5', 'dasdoresfc', 'DM Studio', 'Dom Camillo68', 'FBC Colorado', 'FBC Colorado II', 'FC castelo Branco 2', 'FC Los Castilho', 'Fedato Futebol Clube', 'FIGUEIRA DA ILHA', 'FÚRIA LEON', 'Gig@ntte', 'Gremiomaniasm', 'Grêmio imortal 37', 'JUV. KP', 'JV5 Tricolor Gaúcho', 'LISI GREMISTA', 'lsauer fc', 'Luis lemes inter', 'MAFRA MARTINS FC', 'Mau Humor F.C.', 'mercearia Estrela', 'Máquina Laranjja', 'NaoVaiDescer!', 'Paulo Virgili FC', 'Pity10', 'pra sempre imortal fc', 'Profit Soccer', 'PUXE FC', 'Rolo Compressor ZN', 'S.E.R. GRILLO', 'seralex', 'SERGRILLO', 'Sport Clube PAIM', 'Super Vasco f.c', 'SUPER VASCÃO F.C', 'Tabajara de Inhaua PB1', 'TATITTA FC', 'Tatols Beants F.C', 'TEAM LOPES 99', 'teves_futsal20 f.c', 'Texas Club 2026', 'TIGRE LEON', 'Time do S.A.P.O', 'VASCO MARTINS FC']
Snippet salvo em: partici

In [12]:
JS_PATH = Path("participantesLiga_Classica.js")  # ajuste o caminho se estiver em outra pasta

# extrai o array do snippet: window.participantesLiga = [ ... ];
m = re.search(r'window\.participantesLiga\s*=\s*(\[[\s\S]*?\])\s*;', JS_PATH.read_text(encoding="utf-8"))
if not m:
    raise ValueError(f"Não consegui achar a lista dentro de {JS_PATH}")

nomes_times = json.loads(m.group(1))

# (opcional) dedup + ordena
nomes_times = sorted(dict.fromkeys(nomes_times), key=str.casefold)

print(len(nomes_times), "times")
nomes_times[:10]

52 times


['A Lenda Super Vasco F.c',
 'A Lenda Super Vascão f.c',
 'AZURRA82',
 'Bandoleros FCS',
 'BordonFC04',
 'C.A. Charru@',
 'cartola scheuer17',
 'CARTOLEIRO DO VALLE PRO26.5',
 'dasdoresfc',
 'DM Studio']

In [13]:

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função para buscar o ID de um time pelo nome usando correspondência flexível
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]
        nome_proximo = get_close_matches(nome_time, nomes_api, n=1, cutoff=0.6)
        if nome_proximo:
            for time in times:
                if time.nome == nome_proximo[0]:
                    return time.id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar os IDs dos times pelo nome
for nome in nomes_times:
    time_id = buscar_id_time(nome)
    if time_id:
        ids_times[nome] = time_id
    else:
        print(f"Não foi possível encontrar o ID para o time {nome}")

print(nomes_times)

['A Lenda Super Vasco F.c', 'A Lenda Super Vascão f.c', 'AZURRA82', 'Bandoleros FCS', 'BordonFC04', 'C.A. Charru@', 'cartola scheuer17', 'CARTOLEIRO DO VALLE PRO26.5', 'dasdoresfc', 'DM Studio', 'Dom Camillo68', 'FBC Colorado', 'FBC Colorado II', 'FC castelo Branco 2', 'FC Los Castilho', 'Fedato Futebol Clube', 'FIGUEIRA DA ILHA', 'FÚRIA LEON', 'Gig@ntte', 'Gremiomaniasm', 'Grêmio imortal 37', 'JUV. KP', 'JV5 Tricolor Gaúcho', 'LISI GREMISTA', 'lsauer fc', 'Luis lemes inter', 'MAFRA MARTINS FC', 'Mau Humor F.C.', 'mercearia Estrela', 'Máquina Laranjja', 'NaoVaiDescer!', 'Paulo Virgili FC', 'Pity10', 'pra sempre imortal fc', 'Profit Soccer', 'PUXE FC', 'Rolo Compressor ZN', 'S.E.R. GRILLO', 'seralex', 'SERGRILLO', 'Sport Clube PAIM', 'Super Vasco f.c', 'SUPER VASCÃO F.C', 'Tabajara de Inhaua PB1', 'TATITTA FC', 'Tatols Beants F.C', 'TEAM LOPES 99', 'teves_futsal20 f.c', 'Texas Club 2026', 'TIGRE LEON', 'Time do S.A.P.O', 'VASCO MARTINS FC']


In [14]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_classica.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls)


✅ Arquivo salvo com sucesso: links_times_cartola_liga_classica.xlsx


,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,A Lenda Super Vascão f.c,49355335,https://cartola.globo.com/#!/time/49355335
2,AZURRA82,18346776,https://cartola.globo.com/#!/time/18346776
3,Bandoleros FCS,13913874,https://cartola.globo.com/#!/time/13913874
4,BordonFC04,50988641,https://cartola.globo.com/#!/time/50988641
5,C.A. Charru@,25748736,https://cartola.globo.com/#!/time/25748736
6,cartola scheuer17,3851966,https://cartola.globo.com/#!/time/3851966
7,CARTOLEIRO DO VALLE PRO26.5,49243759,https://cartola.globo.com/#!/time/49243759
8,dasdoresfc,7017989,https://cartola.globo.com/#!/time/7017989
9,DM Studio,387186,https://cartola.globo.com/#!/time/387186


In [15]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual
# rodada_atual = 1

print("Rodada Atual:", rodada_atual)

Rodada Atual: 1


## Obter Pontuações da Rodada

In [16]:
# ===============================
# CARTOLA FC - CLASSIFICACAO COM PARCIAIS DA RODADA (CLASSICA)
# ===============================
import time
import json
import requests
import pandas as pd

try:
    import cartolafc
    from cartolafc import CartolaFCError
except Exception:
    CartolaFCError = Exception

# -------------------------------
# CONFIG
# -------------------------------
CAP_MULT = 1.5
PER_REQ_SLEEP = 0.9
MAX_RETRIES = 3

sess = requests.Session()
sess.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/plain, */*",
})

# -------------------------------
# HELPERS HTTP / API
# -------------------------------
def http_status_e_rodada():
    r = sess.get("https://api.cartola.globo.com/mercado/status", timeout=20)
    r.raise_for_status()
    data = r.json()
    return int(data.get("status_mercado", 0)), int(data.get("rodada_atual", 0))

def get_parciais():
    r = sess.get("https://api.cartola.globo.com/atletas/pontuados", timeout=30)
    r.raise_for_status()
    data = r.json()
    return data.get("atletas", {})

def fetch_time_payload(time_id: int, rodada: int, max_retries=MAX_RETRIES):
    urls = [
        f"https://api.cartola.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartola.globo.com/time/id/{time_id}",
    ]
    for url in urls:
        wait = 0.7
        for attempt in range(1, max_retries + 1):
            try:
                r = sess.get(url, timeout=30)
                status = r.status_code
                if status == 200:
                    d = r.json()
                    if not isinstance(d, dict):
                        d = {}
                    return d
                elif status in (403, 429, 503):
                    time.sleep(wait); wait *= 1.8
                    continue
                else:
                    break
            except Exception:
                time.sleep(wait); wait *= 1.8
        # proxima URL
    return {}


def fetch_partidas_rodada(rodada: int):
    try:
        r = sess.get(f"https://api.cartola.globo.com/partidas/{rodada}", timeout=30)
        if r.status_code != 200:
            return []
        data = r.json()
        if isinstance(data, dict):
            partidas = data.get("partidas", [])
            return partidas if isinstance(partidas, list) else []
    except Exception:
        pass
    return []


def clubes_que_ja_jogaram(rodada: int):
    partidas = fetch_partidas_rodada(rodada)
    clubes = set()
    for p in partidas:
        if not isinstance(p, dict):
            continue
        status = (p.get("status_transmissao_tr") or p.get("status_transmissao") or "").upper()
        encerrada = status in ("ENCERRADA", "FINALIZADA")
        if p.get("placar_oficial_mandante") is not None or p.get("placar_oficial_visitante") is not None:
            encerrada = True
        if encerrada:
            mid = p.get("clube_casa_id")
            vid = p.get("clube_visitante_id")
            try:
                if mid is not None:
                    clubes.add(int(mid))
            except Exception:
                pass
            try:
                if vid is not None:
                    clubes.add(int(vid))
            except Exception:
                pass
    return clubes


def setor_por_posicao(posicao_id: int) -> str:
    mapa = {
        1: "Goleiro",
        2: "Laterais",
        3: "Zagueiros",
        4: "Meias",
        5: "Atacantes",
        6: "Tecnico",
    }
    return mapa.get(int(posicao_id), "")


def _id_int(val):
    try:
        return int(val)
    except Exception:
        return None


def calcular_parcial_time(time_id: int, rodada: int, mapa_pontuados: dict, clubes_jogaram: set):
    data = fetch_time_payload(time_id, rodada)
    atletas = data.get("atletas") if isinstance(data, dict) else None
    if not isinstance(atletas, list):
        return 0.0, [], None

    titulares = []
    tecnico = None
    for a in atletas:
        if not isinstance(a, dict):
            continue
        pos = a.get("posicao_id")
        if pos == 6:
            tecnico = a
        else:
            titulares.append(a)

    luxo_id = data.get("reserva_luxo_id") if isinstance(data, dict) else None
    if luxo_id is not None:
        luxo_id = _id_int(luxo_id)

    # Capitao (1.5x)
    capitao_id = None
    try:
        capitao_id = data.get("capitao_id") if isinstance(data, dict) else None
        if capitao_id is None and isinstance(data.get("time"), dict):
            capitao_id = data["time"].get("capitao_id")
        capitao_id = _id_int(capitao_id)
    except Exception:
        capitao_id = None

    titulares_por_pos = {}
    for a in titulares:
        pos = a.get("posicao_id")
        titulares_por_pos.setdefault(pos, []).append(a)

    atletas_em_jogo = list(titulares)
    subs_banco = []

    reservas = data.get("reservas", []) if isinstance(data, dict) else []
    if isinstance(reservas, list):
        for r in reservas:
            if not isinstance(r, dict):
                continue
            # reserva de luxo nao entra como banco
            if luxo_id is not None and _id_int(r.get("atleta_id")) == luxo_id:
                continue
            # banco so entra se reserva tiver pontuacao parcial positiva
            rid = _id_int(r.get("atleta_id"))
            if rid is None or rid not in mapa_pontuados:
                continue
            if float(mapa_pontuados.get(rid, 0.0)) <= 0.0:
                continue
            pos = r.get("posicao_id")
            if pos is None:
                continue
            candidatos = titulares_por_pos.get(pos, [])
            titular_sub = None
            for t in candidatos:
                tid = _id_int(t.get("atleta_id"))
                if tid is None:
                    continue
                clube_tid = _id_int(t.get("clube_id"))
                if tid not in mapa_pontuados and (clube_tid in clubes_jogaram):
                    titular_sub = t
                    break
            if titular_sub is None:
                continue
            try:
                atletas_em_jogo.remove(titular_sub)
            except ValueError:
                pass
            atletas_em_jogo.append(r)
            subs_banco.append((titular_sub.get("atleta_id"), r.get("atleta_id"), pos))

    # Reserva de luxo
    sub_luxo = None
    luxo_capitao = False
    if luxo_id is not None:
        ids_em_jogo = set()
        for a in atletas_em_jogo:
            ids_em_jogo.add(_id_int(a.get("atleta_id")))
        if luxo_id not in ids_em_jogo:
            luxo_obj = None
            if isinstance(reservas, list):
                for r in reservas:
                    if isinstance(r, dict) and _id_int(r.get("atleta_id")) == luxo_id:
                        luxo_obj = r
                        break
            if isinstance(luxo_obj, dict):
                p_luxo = float(mapa_pontuados.get(luxo_id, 0.0))
                if p_luxo > 0:
                    setor_luxo = setor_por_posicao(luxo_obj.get("posicao_id"))
                    candidatos_setor = []
                    for a in atletas_em_jogo:
                        if setor_por_posicao(a.get("posicao_id")) == setor_luxo:
                            candidatos_setor.append(a)
                    if candidatos_setor:
                        # luxo so entra se TODOS do setor ja jogaram
                        todos_setor_jogaram = True
                        for c in candidatos_setor:
                            clube_c = _id_int(c.get("clube_id"))
                            if clube_c not in clubes_jogaram:
                                todos_setor_jogaram = False
                                break
                        if todos_setor_jogaram:
                            def pts(a):
                                try:
                                    return float(mapa_pontuados.get(int(a.get("atleta_id")), 0.0))
                                except Exception:
                                    return 0.0
                            # pior titular do setor; desempate favorece capitao
                            pior_pts = None
                            piores = []
                            for c in candidatos_setor:
                                v = pts(c)
                                if pior_pts is None or v < pior_pts:
                                    pior_pts = v
                                    piores = [c]
                                elif v == pior_pts:
                                    piores.append(c)
                            pior = None
                            if len(piores) > 1 and capitao_id is not None:
                                for c in piores:
                                    if _id_int(c.get("atleta_id")) == capitao_id:
                                        pior = c
                                        break
                            if pior is None:
                                pior = piores[0] if piores else None
                            if pior is not None and p_luxo > pts(pior):
                                if capitao_id is not None and _id_int(pior.get("atleta_id")) == capitao_id:
                                    luxo_capitao = True
                                try:
                                    atletas_em_jogo.remove(pior)
                                except ValueError:
                                    pass
                                atletas_em_jogo.append(luxo_obj)
                                sub_luxo = (pior.get("atleta_id"), luxo_id, setor_luxo)

    total = 0.0
    for a in atletas_em_jogo:
        try:
            aid = int(a.get("atleta_id"))
        except Exception:
            continue
        total += float(mapa_pontuados.get(aid, 0.0))

    # Bonus do capitao (50%)
    if capitao_id is not None:
        cap_pts = float(mapa_pontuados.get(capitao_id, 0.0))
        if cap_pts != 0.0:
            ids_em_jogo = set()
            for a in atletas_em_jogo:
                ids_em_jogo.add(_id_int(a.get("atleta_id")))
            if capitao_id in ids_em_jogo:
                total += cap_pts * 0.5
            elif luxo_capitao and luxo_id is not None and luxo_id in ids_em_jogo:
                luxo_pts = float(mapa_pontuados.get(luxo_id, 0.0))
                if luxo_pts != 0.0:
                    total += luxo_pts * 0.5

    if isinstance(tecnico, dict):
        try:
            tid = int(tecnico.get("atleta_id"))
            total += float(mapa_pontuados.get(tid, 0.0))
        except Exception:
            pass

    return round(total, 2), subs_banco, sub_luxo

# -------------------------------
# 1) RODADA ATUAL E DF BASE
# -------------------------------
try:
    rodada_atual = api.mercado().rodada_atual
except CartolaFCError:
    rodada_atual = 1


def campeonato_comecou(ids_times: dict) -> bool:
    for time_id in ids_times.values():
        try:
            p = api.time(time_id=int(time_id), rodada=1).ultima_pontuacao
            if p is not None:
                return True
        except Exception:
            continue
    return False

if not campeonato_comecou(ids_times):
    total_rodadas = 38
    df_pontuacoes = pd.DataFrame(index=ids_times.keys(),
                                 columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
    df_pontuacoes[:] = 0.0
else:
    def obter_pontuacao_por_rodada(time_id: int, rodada_atual_api: int) -> dict:
        pontuacoes = {}
        fim_inclusivo = max(1, int(rodada_atual_api))
        for rodada in range(1, fim_inclusivo + 1):
            try:
                t = api.time(time_id=int(time_id), rodada=rodada)
                pontuacoes[rodada] = t.ultima_pontuacao
            except Exception:
                pontuacoes[rodada] = None
        return pontuacoes

    df_pontuacoes = pd.DataFrame()
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(int(time_id), rodada_atual)
        df_pontuacoes[nome] = pd.Series(pontuacoes)

    df_pontuacoes = df_pontuacoes.transpose()
    if rodada_atual > 1:
        df_pontuacoes.columns = [f'Rodada {i}' for i in range(1, rodada_atual + 1)]

    df_pontuacoes = df_pontuacoes.apply(pd.to_numeric, errors="coerce").fillna(0.0)

# -------------------------------
# 2) APLICA PARCIAIS (COM BANCO, LUXO E CAPITAO)
# -------------------------------
status_http, rodada_http = http_status_e_rodada()
rodada_parcial = int(rodada_http) if status_http == 2 else None

print(
    f"Status HTTP={status_http} | rodada_http={rodada_http} | "
    f"rodada_api={rodada_atual} | rodada_parcial={rodada_parcial}"
)

parciais_raw = get_parciais()
mapa_pontuados = {}
if isinstance(parciais_raw, dict):
    for k, v in parciais_raw.items():
        try:
            mapa_pontuados[int(k)] = float(v.get("pontuacao") or 0.0)
        except Exception:
            continue

if not mapa_pontuados:
    print("Sem parciais no momento. Mantendo valores anteriores.")
else:
    if rodada_parcial is None:
        rodada_parcial = int(rodada_atual)

    col_parcial = f"Rodada {rodada_parcial}"
    if col_parcial not in df_pontuacoes.columns:
        df_pontuacoes[col_parcial] = 0.0

    if "Lider_Rodada" in df_pontuacoes.index:
        df_pontuacoes = df_pontuacoes.drop(index="Lider_Rodada", errors="ignore")

    clubes_jogaram = clubes_que_ja_jogaram(rodada_parcial)

    print("Rodada em andamento: consultando parciais")
    atualizados, relatorio = 0, []
    for nome_time, time_id in ids_times.items():
        try:
            time_id = int(time_id)
            total, _, _ = calcular_parcial_time(time_id, rodada_parcial, mapa_pontuados, clubes_jogaram)
            df_pontuacoes.loc[nome_time, col_parcial] = round(total, 2)
            atualizados += 1
            relatorio.append((nome_time, round(total, 2), "ok"))
            time.sleep(PER_REQ_SLEEP)
        except Exception as e:
            relatorio.append((nome_time, 0.0, f"erro:{type(e).__name__}"))

    print(f"Times atualizados: {atualizados} / {len(ids_times)} na coluna '{col_parcial}'")
    for row in relatorio[:12]:
        nm, tot, mot = row
        print(f" - {nm:25} | total={tot:6.2f} | {mot}")

# Recalcula marcador de lider por rodada (opcional, so se houver algum > 0)
try:
    df_num = df_pontuacoes.apply(pd.to_numeric, errors="coerce")
    if (df_num.sum(numeric_only=True).sum() or 0) != 0:
        lider_rodada = df_num.idxmax()
        df_pontuacoes.loc["Lider_Rodada"] = lider_rodada
except Exception as e:
    print("Nao foi possivel recalcular 'Lider_Rodada':", e)

# Normaliza tipos
df_pontuacoes = df_pontuacoes.apply(pd.to_numeric, errors="coerce").fillna(0.0)

# -------------------------------
# 3) SEGMENTACAO POR TURNOS E MESES
# -------------------------------
rodadas_por_mes = {
    "Janeiro":   [1],
    "Fevereiro": [2, 3, 4],
    "Marco":  [5, 6, 7, 8],
    "Abril": [9, 10, 11, 12, 13],
    "Maio": [14, 15, 16, 17, 18],
    "Julho": [19, 20, 21]
}
rodadas_segundo_turno = list(range(20, 39))

def somar_pontuacoes(df, rodadas):
    colunas = [f'Rodada {i}' for i in rodadas if f'Rodada {i}' in df.columns]
    return df[colunas].drop(index='Lider_Rodada', errors='ignore').sum(axis=1).sort_values(ascending=False)

df_turno_2 = df_pontuacoes[[f'Rodada {i}' for i in rodadas_segundo_turno if f'Rodada {i}' in df_pontuacoes.columns]]
df_soma_turno_2 = somar_pontuacoes(df_pontuacoes, rodadas_segundo_turno)
df_soma_mensal = {mes: somar_pontuacoes(df_pontuacoes, rodadas) for mes, rodadas in rodadas_por_mes.items()}

# -------------------------------
# 4) SALVAR EXCEL E JS
# -------------------------------
with pd.ExcelWriter("Pontuacoes_Liga_Classica_Completa.xlsx") as writer:
    df_pontuacoes.to_excel(writer, sheet_name="Geral")
    df_turno_2.to_excel(writer, sheet_name="Turno 2")
    pd.DataFrame(df_soma_turno_2).to_excel(writer, sheet_name="Classif Turno 2")
    for mes, serie in df_soma_mensal.items():
        pd.DataFrame(serie).to_excel(writer, sheet_name=f"Mes - {mes}")

print("Excel: Pontuacoes_Liga_Classica_Completa.xlsx salvo.")

# JS
df_dict_geral = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore').fillna(0).to_dict(orient="index")
df_js = {
    "geral": df_dict_geral,
    "turnos": {"turno_2": df_soma_turno_2.to_dict()},
    "meses": {mes: serie.to_dict() for mes, serie in df_soma_mensal.items()}
}
js_path = "Pontuacoes_Liga_Classica.js"

# Parcial da rodada atual (JS)
try:
    col_parcial = f"Rodada {rodada_parcial}" if rodada_parcial is not None else None
    parcial_payload = {
        "rodada": int(rodada_parcial) if rodada_parcial is not None else int(rodada_atual),
        "times": {
            str(ids_times[nome]): float(df_pontuacoes.at[nome, col_parcial])
            for nome in df_pontuacoes.index
            if col_parcial and nome in ids_times and str(df_pontuacoes.at[nome, col_parcial]) not in ("", "nan")
        }
    }
except Exception:
    parcial_payload = {"rodada": int(rodada_atual), "times": {}}

parcial_js = f"const pontuacaoParcialRodadaAtual = {json.dumps(parcial_payload, indent=2, ensure_ascii=False)};"

with open(js_path, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoLigaClassica = {json.dumps(df_js, indent=2, ensure_ascii=False)};")
    meta = {"status_http": int(status_http), "rodada_parcial": (int(rodada_parcial) if rodada_parcial is not None else None)}
    f.write("window.ligaClassicaMeta = " + json.dumps(meta) + ";")
    f.write("\n" + parcial_js)

print(f"JS: {js_path} salvo.")


Status HTTP=2 | rodada_http=1 | rodada_api=1 | rodada_parcial=1
Rodada em andamento: consultando parciais
Times atualizados: 52 / 52 na coluna 'Rodada 1'
 - A Lenda Super Vasco F.c   | total= 59.56 | ok
 - A Lenda Super Vascão f.c  | total= 45.30 | ok
 - AZURRA82                  | total= 30.06 | ok
 - Bandoleros FCS            | total= 69.56 | ok
 - BordonFC04                | total= 54.06 | ok
 - C.A. Charru@              | total= 71.71 | ok
 - cartola scheuer17         | total= 53.06 | ok
 - CARTOLEIRO DO VALLE PRO26.5 | total= 69.26 | ok
 - dasdoresfc                | total= 71.36 | ok
 - DM Studio                 | total= 47.20 | ok
 - Dom Camillo68             | total= 58.40 | ok
 - FBC Colorado              | total= 45.46 | ok
Excel: Pontuacoes_Liga_Classica_Completa.xlsx salvo.
JS: Pontuacoes_Liga_Classica.js salvo.


In [17]:
# Exibir a pontuação
display(df_pontuacoes)

,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19,Rodada 20,Rodada 21,Rodada 22,Rodada 23,Rodada 24,Rodada 25,Rodada 26,Rodada 27,Rodada 28,Rodada 29,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38
A Lenda Super Vasco F.c,59.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A Lenda Super Vascão f.c,45.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AZURRA82,30.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Bandoleros FCS,69.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BordonFC04,54.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C.A. Charru@,71.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
cartola scheuer17,53.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
CARTOLEIRO DO VALLE PRO26.5,69.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
dasdoresfc,71.36,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
DM Studio,47.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
